# Practical 2: Mobile Robots

Let's set the variable, *running_locally*, again according to your setup. If you are running the notebook on your local machine, set *running_locally* to true. If you are running the notebook on colab, set *running_locally* to false. Note: visualization will be disabled on colab.

In [ ]:
running_locally = True

We will import the libraries in the following blocks.

In [ ]:
import sys
import os
if running_locally:
  from os import listdir
  from os.path import isfile, join
  sys.path.append('Practical2_Support')
  onlyfiles = [f for f in listdir('Practical2_Support') if isfile(join('Practical2_Support', f))]
  print(onlyfiles)
  import matplotlib.pyplot as plt
else:
  !pip install jupyter-ui-poll
  from jupyter_ui_poll import ui_events
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  sys.path.append('/content/gdrive/MyDrive/ECE4078_Practical/Week02/Practical2_Support')
  !ls '/content/gdrive/MyDrive/ECE4078_Practical/Week02/Practical2_Support'
  import plotly.graph_objects as plt 
  from Colab_Renderer import *

In [ ]:
#Import all the necessary modules
%matplotlib inline
import numpy as np
# widgest allows interactive elements 
import ipywidgets as widgets
import pickle
from IPython.display import clear_output
# Visualizer
from Renderer import *

import time

# 1. Bicycle Kinematic Model

The configuration (also known as state) of the bycycle is descibed by its 2D position and orientation, i.e., $(x, y, \theta)$. The control inputs are the speed at the rear wheel and the steering angle at the front wheel, i.e., $(v, \delta)$

In the following we implement the bicycle kinematic model and show how its motion is described in the world frame.

In [ ]:
# Define Bicycle model

class Bicycle(object):
    """Implementation of the kinematic bicycle model (rear wheel model)
    length = distance (in meters) between the wheels
    gamma_max = maximum steering angle
    speed_max = maximum speed
    """

    def __init__(self, length=1, delta_max=np.pi/3, vel_max=5):
        
        # Length between the wheels
        self.length = length
        
        # Maximum velocity and steering angle
        self.delta_max = delta_max
        self.vel_max = vel_max
        
        # Configuration of bicycle in world frame, x,y = coordinates in world frame, theta=orientation
        self.x = 0                      
        self.y = 0                      
        self.theta = 0                  
        self.states = []
        
        # Initial control inputs
        self.velocity = 0
        self.steering = 0
           
                
    def update_control(self, v, delta):
        self.velocity = np.clip(v, -self.vel_max, self.vel_max)
        self.delta = np.clip(delta, -self.delta_max, self.delta_max)
           
    def reset(self):
        first_state = self.states[0]
        self.x, self.y, self.theta = first_state
        del self.states[:]
        self.states = [first_state]
    
    def get_state(self):
        """Return the current bicycle state. The state is in (x,y,theta) format"""
        return (self.x, self.y, self.theta)
    
    def set_state(self,x=0,y=0,theta=0):
        """Sets the model new state"""
        self.x = x
        self.y = y
        self.theta = theta
        self.states.append((x, y, theta))
    
    def drive(self, dt=0.01):
        """
        Update the bicycle's state.
        """        
        # Compute velocity using control input and current orientation
        v_x = self.velocity * np.cos(self.theta)
        v_y = self.velocity * np.sin(self.theta)
        
        # Compute next position
        next_x = self.x + v_x * dt
        next_y = self.y + v_y * dt
                
        # Compute angular velocity given control input (velocity and
        # steering angle)
        omega = self.velocity * np.tan(self.delta) / self.length
        
        # Compute next orientation
        next_theta = self.theta + omega * dt
        self.set_state(next_x, next_y, next_theta)

## Lets now test driving the bicycle
- Use the slider to adjust the velocity and steering angle. You may do so in both versions of the notebook (local instal or colab). However, there will be no live visualizationn for the colab version. Instead, you will see text output changing when you change the slider and you will see the final trajectory displayed.
- Currently the animation will play for 60 frames. You can adjust this number by providing the ```max_iterations``` argument to the function call ```rend.initialize(bot, max_iterations=[integer])```

In [ ]:
# Test driving the robot
# some parameters:
max_iterations = 60 # maximum number of animation frames
max_vel = 5         # robot's maximum velocity
max_delta = np.pi/3 # robot's maximum steering angle
bike_length = 0.25  # create a bicycle with .25 m between the rear and front wheel
robot_start_x = -2.8 # robot's starting x position
robot_start_y = -2.8 # robot's starting y position
robot_start_theta = np.pi/6 # robot's starting orientation

bot = Bicycle(bike_length, max_delta, max_vel)

bot.set_state(robot_start_x, robot_start_y, robot_start_theta)


# Create the widgets
v_slider = widgets.FloatSlider(value=0, min=-max_vel, max=max_vel, step=0.1, 
                                              description=r'Velocity (v)',
                                              continuous_update=False, orientation='horizontal',
                                              readout=True, readout_format='.1f')
delta_slider = widgets.FloatSlider(value=0, min=-max_delta, max=max_delta, step=0.05, 
                                                  description=r'Angle (delta)', 
                                                  continuous_update=False, orientation='horizontal',
                                                  readout=True, readout_format='.1f')



ui = widgets.HBox([v_slider, delta_slider])

if running_locally:
  control_widgets = widgets.interactive_output(bot.update_control, {'v': v_slider, 'delta': delta_slider})
  display(ui, control_widgets)

  %matplotlib notebook
  # call the graphics function if running locally
  rend = Renderer.Instance()
  rend.initialize(bot, dt_data=0.02, max_iterations=max_iterations, title_txt='Figure 1 - Overhead View')
else:
  import plotly.graph_objects as plt 
  # only console output is available on colab 
  # the following declares the functions and widgets
  ui_done = False
  
  def reset_interface(reset_btn):
      clear_output(wait=True)
      display(ui, ui_buttons)
      print('Live visualization has been disabled on colab')
      global ui_done
      ui_done = False
      bot.set_state(robot_start_x, robot_start_y, robot_start_theta)

  def on_click(btn):
      global ui_done
      ui_done = True
      print('Stopped. Please reset.')      

  def loop_event(start_btn):
      with ui_events() as poll:
          robot_traj = []
          for i in range(0, max_iterations):
              if ui_done==False:
                  poll(10)
                  print('frame: %i, x: %.2f, y: %.2f, theta: %.2f' %(i, bot.x, bot.y, bot.theta))
                  robot_traj.append([bot.x, bot.y])
                  bot.update_control(v_slider.value, delta_slider.value)
                  bot.drive()
                  time.sleep(0.2)
              else:
                  break
          
          clear_output(wait=True)
          display(ui, ui_buttons)
          plot_traj(robot_traj, 'Figure 1 - Overhead View')
      
  stop_btn = widgets.Button(description='Stop')
  start_btn = widgets.Button(description='Start')
  reset_btn = widgets.Button(description='Reset')
  stop_btn.on_click(on_click)
  start_btn.on_click(loop_event)
  reset_btn.on_click(reset_interface)
  ui_buttons = widgets.HBox([start_btn, stop_btn, reset_btn])
  display(ui, ui_buttons)
  print('Live visualization has been disabled on colab')

# 2. PenguinPi Kinematic Model

<img src=https://cirrusrobotics.com.au/images/products/penguinpi/front_a.jpg width="340" height="300">


The state of the robot is described by its 2D position $(x,y)$  and orientation $\theta$. Our control inputs are the linear and angular velocity of the robot, i.e., $v$ and $\omega$. 

We will define the <code>drive(self, dt=0.02)</code> function that will update the state of the robot given a sequence of control inputs.

**Interaction**
- Move the sliders to change the control inputs

**TODO**
- Find the coding error 
- Complete the code that makes the robot turn

In [ ]:
class PenguinPi(object):
    
    def __init__(self, max_v=5, max_omega=np.pi):
        
        # Robot state
        self.x = 0
        self.y = 0
        self.theta = 0
        
        # List of all PenguiPi states in current simulation and wheel speeds
        self.states = []
        
        # Control inputs
        self.linear_velocity = 0
        self.angular_velocity = 0
        
        # Control input bounds
        self.max_linear_velocity = max_v
        self.max_angular_velocity = max_omega
                
                          
    def drive(self, dt=0.02):
        """
         Update the PenguiPi state
        """        
        
        # Remember that the PenguiPi current state is given by self.x, self.y, self.theta        
        # Apply the velocities
        if self.angular_velocity == 0:
            next_x = self.x - np.cos(self.theta)*self.linear_velocity*dt
            next_y = self.y + np.sin(self.theta)*self.linear_velocity*dt
            next_theta = self.theta
        else:
#---------------------------TODO: Complete code that make the robot turn--------------
            R = 
            next_theta = 
            next_x = 
            next_y = 
#--------------------------------------------------------------------------------------
       
        # Make next state our current state
        self.set_state(next_x, next_y, next_theta)        
        
    def update_control(self, linear_v, angular_v):
        # We read the values from the sliders and update the robot's linear and angular velocity
        self.linear_velocity = np.clip(linear_v, -self.max_linear_velocity, self.max_linear_velocity)
        self.angular_velocity = np.clip(angular_v, -self.max_angular_velocity , self.max_angular_velocity)
           
    def reset(self):
        # We bring the robot to its initial state and set control to default values
        # This method is called after a click in the "reset" button or when the max. number of display 
        # iterations is reached
        first_state = self.states[0]
        self.x, self.y, self.theta = first_state
        del self.states[:]
        self.states = [first_state]
    
    def get_state(self):
        """Return the current robot state. The state is in (x,y,theta) format"""
        return (self.x, self.y, self.theta)
    
    def set_state(self,x=0,y=0,theta=0):
        """Define the new model state"""
        self.x = x
        self.y = y
        self.theta = theta
        self.states.append((x, y, theta))

## Lets now test driving the PenguiPi
- Use the slider to adjust the velocity and steering angle
- Currently the animation will play for 60 frames. You can adjust this number by providing the ```max_iterations``` argument to the function call ```rend.initialize(bot, max_iterations=[integer])```

In [ ]:
# Test driving the robot
# Creater a new PenguinPi object
# some parameters:
max_iterations = 60 # maximum number of animation frames
max_linear_vel = 5
max_angular_vel = np.pi
pi_bot = PenguinPi(max_linear_vel, max_angular_vel)

# Place the robot at -3.0,-3.0 
pibot_start_x = -3.0 # pibot's starting x position
pibot_start_y = -3.0 # pibot's starting y position
pibot_start_theta = np.pi/6 # pibot's starting orientation
pi_bot.set_state(pibot_start_x, pibot_start_y, pibot_start_theta)

# Create widgets to adjust control inputs during simulation        
linear_slider = widgets.FloatSlider(value=0, min=-max_linear_vel, 
                                                 max=max_linear_vel, step=0.1, 
                                                 description=r'v', continuous_update=False, 
                                                 orientation='horizontal', readout=True, readout_format='.1f')
angular_slider = widgets.FloatSlider(value=0, min=-max_angular_vel, 
                                                  max=max_angular_vel, step=0.1, 
                                                  description=r'omega', 
                                                  continuous_update=False, orientation='horizontal',
                                                  readout=True, readout_format='.1f')

if running_locally:
    ui = widgets.HBox([linear_slider, angular_slider])
            
    control_widgets = widgets.interactive_output(pi_bot.update_control, {'linear_v': linear_slider, 
                                                                                    'angular_v': angular_slider})
    display(ui, control_widgets)

    # Instantiate rendenring object
    rend = Renderer.Instance()
    rend.initialize(pi_bot, dt_data=0.02, max_iterations=max_iterations, title_txt='Figure 2 - Overhead View')
else:
    ui_done = False
    def reset_interface(reset_btn):
        clear_output(wait=True)
        display(ui, ui_buttons)
        global ui_done
        ui_done = False
        print('Live visualization has been disabled on colab')
        pi_bot.set_state(pibot_start_x, pibot_start_y, pibot_start_theta)

    def on_click(btn):
        global ui_done
        ui_done = True

    def loop_event(start_btn):
        with ui_events() as poll:
            robot_traj = []
            for i in range(0, max_iterations):
                if ui_done==False:
                    poll(10)
                    print('frame: %i, x: %.2f, y: %.2f, theta: %.2f' %(i, pi_bot.x, pi_bot.y, pi_bot.theta))
                    robot_traj.append([pi_bot.x, pi_bot.y])
                    pi_bot.update_control(v_slider.value, delta_slider.value)
                    pi_bot.drive()
                    time.sleep(0.2)
                else:
                    break
            clear_output(wait=True)
            display(ui, ui_buttons)
            plot_traj(robot_traj, 'Figure 2 - Overhead View')
        
    stop_btn = widgets.Button(description='Stop')
    start_btn = widgets.Button(description='Start')
    reset_btn = widgets.Button(description='Reset')
    stop_btn.on_click(on_click)
    start_btn.on_click(loop_event)
    reset_btn.on_click(reset_interface)
    ui_buttons = widgets.HBox([start_btn, stop_btn, reset_btn])
    display(ui, ui_buttons)
    print('Live visualization has been disabled on colab')